In [37]:
import pandas as pd
import json
import numpy as np
pd.set_option('display.max_rows', None)

In [ ]:
orders = pd.read_csv('true.csv', header=None, quotechar="'")
orders = orders.rename(columns={0: "order_id", 1: "owner_name", 2:"is_developer", 3:"time_created", 4:"items"})
orders.head()

### Creating "Item_Type" Column

In [3]:
items = []
for i in range(len(orders)):
    items.append([])

In [4]:
for i in np.arange(len(orders)):
    item_dict = json.loads(orders['items'][i])
    for nickname, item in item_dict.items():
        item_type = item['type']
        items[i].append(item_type)

In [5]:
orders['item_type'] = items

### Creating 'requirements' column

In [6]:
reqs = []
for i in range(len(orders)):
    reqs.append([])

In [7]:
for i in np.arange(len(orders)):
    item_dict = json.loads(orders['items'][i])
    for nickname, item in item_dict.items():
        reqs[i].append(item_dict[nickname]['requirements'])

In [8]:
orders['requirements'] = reqs

In [11]:
item_set = set()
for i in orders['item_type']:
    for j in i:
        item_set.add(j)
item_set

{'cdm_cluster',
 'cdm_node',
 'config',
 'cookbook_item',
 'esx_host',
 'grafana_node',
 'ip_address',
 'mssql_server',
 'nfs_ds_vm',
 'release_qual_baton',
 'rktest_yml',
 's3_bucket',
 'sd_dev_machine',
 'ubuntu_machine',
 'vm_machine'}

In [12]:
order_cnt = len(orders)
for i in item_set:
    count = 0
    for j in orders['items']:
        if i in j:
            count+=1
    print(i, ' : ', count/order_cnt*100, "%")

config  :  28.189666681500604 %
cdm_cluster  :  37.85990832628722 %
grafana_node  :  0.09567887499443728 %
ubuntu_machine  :  1.766721552222865 %
vm_machine  :  13.036802990521116 %
s3_bucket  :  0.031151261626095855 %
ip_address  :  1.5442125406078946 %
release_qual_baton  :  0.01112545058074852 %
cookbook_item  :  0.24475991277646747 %
cdm_node  :  0.08232833429753905 %
esx_host  :  5.148858528770415 %
nfs_ds_vm  :  0.01780072092919763 %
sd_dev_machine  :  45.47194161363535 %
rktest_yml  :  4.73944194739887 %
mssql_server  :  0.1646566685950781 %


## Looking at cdm_cluster

In [13]:
booly = []
for i in orders['item_type']:
    if 'cdm_cluster' in i:
        booly.append(True)
    else:
        booly.append(False)

In [ ]:
cdm_clust = orders[booly]
cdm_clust.head()

In [ ]:
cdm_clust['item_str'] = cdm_clust['item_type'].apply(', '.join)

In [16]:
dist = (cdm_clust.groupby('item_str').size().to_frame()/len(cdm_clust)*100).sort_values(0, ascending=False)
dist.head()

,0
item_str,
cdm_cluster,84.107206
"cdm_cluster, vm_machine",2.368638
"sd_dev_machine, cdm_cluster",1.545786
"cdm_cluster, cdm_cluster",1.363583
"sd_dev_machine, cdm_cluster, vm_machine, vm_machine",1.181380


In [17]:
cdm_clust_list = ['cdm_cluster']*len(cdm_clust)

In [ ]:
cdm_clust['item_type_specific'] = cdm_clust_list

In [19]:
cdm_clust = cdm_clust.reset_index(drop=True)

In [21]:
cdm_reqs = []
for i in range(len(cdm_clust['requirements'])):
    index = cdm_clust['item_type'][i].index('cdm_cluster')
    dicty = cdm_clust['requirements'][i][index]
    cdm_reqs.append(dicty)

In [22]:
cdm_clust['cdm_reqs'] = cdm_reqs

Below is code to parse through order requirements previously stored in a dictionary

In [23]:
cdm_clust_temp = cdm_clust[['order_id','time_created','item_type_specific', 'cdm_reqs']]

In [24]:
flatten = cdm_clust_temp.cdm_reqs.apply(pd.Series)

In [25]:
cdm_clust = pd.concat([cdm_clust_temp[['order_id', 'time_created','item_type_specific']], flatten], axis=1)

These are all the possible requirement for this type of cluster and the percentage of time they are left blank when placing an order.

In [27]:
# Percent Null

for i in list(cdm_clust.columns[3:]):
    percent = sum(cdm_clust[i].isnull())/(len(cdm_clust))*100
    print(i, " : ", percent)

artifacts_url  :  89.35582461502293
location  :  0.0
model  :  0.0
network  :  49.54743152697778
node_count  :  0.0
has_ipv6  :  0.0
test_mode  :  84.07194075467262
low_latency  :  71.44704361114377
version  :  94.7161161396497
disk_size  :  99.7648994945339
disk_type  :  99.75902198189726
provider  :  15.910426707417422
form  :  15.910426707417422
mode  :  15.910426707417422
tags  :  15.910426707417422
software  :  15.910426707417422
nodes  :  15.910426707417422
old_requirements  :  15.910426707417422
drs  :  99.81191959562713


In [28]:
cdm_clust['location'] = cdm_clust['location'].str.lower()

### Specified Nodes and Old Requirements

In [ ]:
spec = cdm_clust[(cdm_clust.drs.notnull()) & (cdm_clust.old_requirements.notnull()) & (cdm_clust.tags.notnull())]
spec.head()

In [59]:
grouped = spec.groupby(['artifacts_url','location', 'network', 'node_count', 'low_latency', 'has_ipv6', 'model', 'form', 'provider', 'mode', 'test_mode', 'version', 'disk_size', 'disk_type', 'software', 'drs'], dropna=False).size().to_frame()

In [ ]:
percents = grouped/grouped[0].sum()*100
percents.sort_values(by=0, ascending=False).head(10)

### No Old_Requirements or Nodes

In [64]:
nospec = cdm_clust[(cdm_clust.drs.isnull()) & (cdm_clust.old_requirements.isnull()) & (cdm_clust.tags.isnull())]

In [65]:
grouped = nospec.groupby(['artifacts_url','location', 'network', 'node_count', 'low_latency', 'has_ipv6', 'model', 'form', 'provider', 'mode', 'test_mode', 'version', 'disk_size', 'disk_type', 'software', 'drs'], dropna=False).size().to_frame()

In [ ]:
percents = grouped/grouped[0].sum()*100
percents.sort_values(by=0, ascending=False).head(10)

### All Data

In [56]:
grouped = cdm_clust.groupby(['artifacts_url','location', 'network', 'node_count', 'low_latency', 'has_ipv6', 'model', 'form', 'provider', 'mode', 'test_mode', 'version', 'disk_size', 'disk_type', 'software', 'drs'], dropna=False).size().to_frame()

In [57]:
percents = grouped/grouped[0].sum()*100
percents.sort_values(by=0, ascending=False).head(10)

0
artifacts_url location network node_count low_latency has_ipv6 model                  form    provider mode test_mode version disk_size disk_type software                  drs           
NaN           colo     NaN     1          NaN         False    justvm-vmware-standard virtual justvm   test NaN       NaN     NaN       NaN       released_version://master NaN  24.009639
                       native  1          NaN         False    justvm-vmware-standard virtual justvm   test NaN       NaN     NaN       NaN       released_version://master NaN   7.382156
                                          True        False    justvm-vmware-standard NaN     NaN      NaN  True      master  NaN       NaN       NaN                       NaN   3.685200
                       NaN     1          False       False    justvm-vmware-standard virtual justvm   test NaN       NaN     NaN       NaN       released_version://master NaN   2.862349
                                          NaN         False    justvm-vmware-standard virtual justvm   test NaN       NaN     NaN       NaN       released_version://5.3    NaN   2.439168
                                                                                                                                                  released_version://5.2    NaN   2.157047
                       native  1          NaN         False    justvm-vmware-standard virtual justvm   test NaN       NaN     NaN       NaN       released_version://5.3    NaN   1.927824
                                          False       False    justvm-vmware-standard virtual justvm   test NaN       NaN     NaN       NaN       released_version://master NaN   1.216645
                                          NaN         False    justvm-vmware-standard virtual justvm   test NaN       NaN     NaN       NaN       released_version://5.2    NaN   1.116727
                                          True        False    justvm-vmware-standard NaN     NaN      NaN  True      5.2     NaN       NaN       NaN                       NaN   0.893382

In [32]:
grouped2 = cdm_clust.groupby(['location', 'node_count', 'has_ipv6', 'model'], dropna=False).size().to_frame()
percents = grouped2/grouped2[0].sum()*100
percents.sort_values(by=0, ascending=False).head(10)

0
location      node_count has_ipv6 model                            
colo          1          False    justvm-vmware-standard  66.956624
              3          False    justvm-vmware-standard  21.470554
              4          False    justvm-vmware-standard   2.456800
                                  prod_brik                1.869049
aws-us-west-2 3          False    aws-m4.xlarge            1.287175
              1          False    aws-m4.xlarge            1.069707
                                  aws-m5.4xlarge           0.863994
              4          False    aws-m5.4xlarge           0.464323
colo          1          False    justvm-vmware-edge       0.440813
              3          True     justvm-vmware-standard   0.335018

In [33]:
grouped3 = cdm_clust.groupby(['node_count'], dropna=False).size().to_frame()
percents = grouped3/grouped3[0].sum()*100
percents.sort_values(by=0, ascending=False)

,0
node_count,
1,70.048196
3,24.232985
4,5.242741
6,0.211590
5,0.164570
8,0.070530
12,0.017633
32,0.005878
,0.005878


In [34]:
grouped4 = cdm_clust.groupby(['node_count', 'model'], dropna=False).size().to_frame()
percents = grouped4/grouped4[0].sum()*100
percents.sort_values(by=0, ascending=False).head(10)

0
node_count model                              
1          justvm-vmware-standard    67.238745
3          justvm-vmware-standard    21.811449
4          justvm-vmware-standard     2.462678
           prod_brik                  1.874927
3          aws-m4.xlarge              1.298930
1          aws-m4.xlarge              1.093217
           aws-m5.4xlarge             0.863994
4          aws-m5.4xlarge             0.464323
1          justvm-vmware-edge         0.440813
3          justvm-vmware-rvc-medium   0.311508